# predict on sequence

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

In [ ]:
# WE TRAIN A MODEL 
from optimize_forest import quickloaddata as load
import simushape_nostruct as sn
import simushape as ss
data = load()

print len(data['Group1T'][0])
print ss.getXY(data,['Group1T'])

In [ ]:
import xgboost as xgb
reg = xgb.XGBRegressor(**{'reg_alpha': 0.8500639221640397, 'learning_rate': 0.01766263588148119, 'max_delta_step': 1, 'min_child_weight': 3, 'n_estimators': 127, 'reg_lambda': 0.70393814277000044, 'max_depth': 14, 'gamma':0, 'booster': 'dart'})


model = ss.make_model(data, data.keys(),model=reg)


In [ ]:
# WE PREDICT

import simushape as ss
import notebook_helper as helper
test_fasta = ss.read_fasta('data/eval_rrna/galaxy31.fasta')
test_react = ss.read_react('data/eval_rrna/galaxy31.react')



res=[]
for k,v in test_fasta:
    if len(v) < 130:
        res.append((k,v,sn.predict(model,v,150)))
        
if True:
    for name, seq, react in res:
        print name
        INVIVO=test_react[name]
        helper.draw_seq_rea(seq,[INVIVO,react])

    



# DRAW FUNCTION FOR THE SHAPE DATA

In [ ]:
      
dic = {name:react for name, seq, react in res}    
ss.dump_shape(dic, "blabla.react")